# FANDANGO


# Group 1:
Zachary Wilkerson, Tara Bode, Hankun Li

### Some things we want to know:
- Of the Movies Now Playing, how highly rated were they by critics and audience members?
- Of the reviews from audience members, how many occurrences of keywords are there of each review and to what extent (i.e. extra positive vs. positive)?

In [1]:
import pandas as pd
import time # needed for time.sleep
from selenium import webdriver
# new driver (opens browser window)
driver = webdriver.Chrome(r'/Users/Tara/Desktop/Summer A 2021/ACG7849- Web Crawling/selenium_files/chromedriver')
driver.get("https://www.fandango.com/movie-reviews")

In [ ]:
with open(r'/Users/Tara/Desktop/Summer A 2021/ACG7849- Web Crawling/selenium_files/jquery.js', 'r') as jquery_js:
    driver.execute_script( jquery_js.read() )

In [ ]:
# For jquery debugger
'''
var script = document.createElement("script")
script.type = "text/javascript";
script.src = "https://code.jquery.com/jquery-3.6.0.js";
document.getElementsByTagName("head")[0].appendChild(script);
'''

In [ ]:
# figure out the jquery to get all links from the main page
# do this in the debugger
#javascript (browser)
$("tbody.mrp__table--mobile-block a")

In [ ]:
#javascript
#javascript (browser)
els =$("tbody.mrp__table--mobile-block a").filter(function(index){return (index % 4 == 0);})

all_links = []
els.each( function(i,el) { all_links.push( el.href )  }   )

In [ ]:
# write a function to get all links into python/selenium
# to get the href of an element we don't need jquery, just el.href (not $(el).href )
myJs = '''
els =$("tbody.mrp__table--mobile-block a").filter(function(index){return (index % 4 == 0);})

all_links = []
els.each( function(i,el) { all_links.push( el.href )  }   )
return all_links;
'''
links = driver.execute_script( myJs )

In [ ]:
links[0:10]

In [ ]:
driver.get(links[0])

In [ ]:
# figure out the jquery to get the information from each project page
section with class ratings-stats holds the information
each cell within that with class holds one type of information
of that cell .slice(0,3) has the critic reviews, .slice(3,6) has the audience reviews

#javascript (browser)
els = $("section.movie-reviews")
        
then for each of these, get criticReviews and audienceReviews
$(els[0]).find("span.movie-ratings-tabs__rating").text().slice(0,3)
$(els[0]).find("span.movie-ratings-tabs__rating").text().slice(3,6)

In [ ]:
# write a function to get the info into python/selenium

# 16 cells that have h5 (number), and h6 (description) element
cells = $("section.movie-reviews")


details = {}

cells.each( 
    function(index, el) {  
        criticReviews = $( el ).find("span.movie-ratings-tabs__rating").text().slice(0,3)
        audienceReviews = $( el ).find("span.movie-ratings-tabs__rating").text().slice(3,6)
        if (audienceReviews != "") { 
            details[audienceReviews.trim()] = criticReviews.trim();
        }               
    }      
)

In [ ]:
myJs_details ='''
cells = els = $("section.movie-reviews")

details = {}

cells.each( 
    function(index, el) {  
        criticReviews = $( el ).find("span.movie-ratings-tabs__rating").text().slice(0,3)
        audienceReviews = $( el ).find("span.movie-ratings-tabs__rating").text().slice(3,6)
        if (criticReviews != "") { 
            details[criticReviews.trim()] = audienceReviews.trim();
        }               
    }      
)
return details;

'''
d = driver.execute_script( myJs_details )
d

In [ ]:
# list with all details
all_details = []

# wrap everything into a loop (get all links, visit each page, get info)
for link in links[0:5]:
    print ("Retrieving reviews from " + link)
    # go to url
    driver.get( link  )
    # some time to finish loading
    driver.implicitly_wait(1)
    # normally inject jquery, but it is already there
    # skip
    # get details
    d = driver.execute_script( myJs_details )
    d["url"] = link    
    print('details', d)
    all_details.append ( d )  

In [ ]:
all_details

In [ ]:
#### All Below is Original For Loop

In [ ]:
# create keyword lists
positive = ['love','fun','good','quality','great']
extra_positive = ['phenomenal','top','heartwarming','favorite','awesome']

negative = [ 'horrible', 'predictable', 'boring', 'bad']
extra_negative = [ 'disappointing', 'worst', 'waste', 'awful','terrible']

In [ ]:
movie_list = []
critic_score_list = []
audience_score_list = []
positiveReviews = []
extraPositiveReviews = []
negativeReviews = []
extraNegativeReviews = []

for item in range(len(pagesToCrawl)):
    positiveReviewsCnt = 0
    extraPositiveReviewsCnt = 0
    negativeReviewsCnt = 0
    extraNegativeReviewsCnt = 0
    
    driver.get(pagesToCrawl[item])
    time.sleep(5)
    try:
        critic_score = driver.find_elements_by_class_name('movie-ratings-tabs__rating')[0].text
        audience_score = driver.find_elements_by_class_name('movie-ratings-tabs__rating')[1].text
        movie_list.append(driver.find_element_by_xpath('//*[@id="subnav"]/div/div/h1/a').get_attribute('data-name'));
        critic_score_list.append(critic_score);
        audience_score_list.append(audience_score);
        review_list = driver.find_elements_by_class_name('rt-audience-reviews__content')
        for review in review_list:
            text = review.text
            for word in positive: 
                positiveReviewsCnt += text.count(word)
            for word in extra_positive: 
                extraPositiveReviewsCnt += text.count(word)
            for word in negative: 
                negativeReviewsCnt += text.count(word) 
            for word in extra_negative: 
                extraNegativeReviewsCnt += text.count(word)
        positiveReviews.append(positiveReviewsCnt)
        extraPositiveReviews.append(extraPositiveReviewsCnt)
        negativeReviews.append(negativeReviewsCnt)
        extraNegativeReviews.append(extraNegativeReviewsCnt)
    except:
        print('No rating score to display for '+pagesToCrawl[item]+' . . . \n')
        movie_list.append(driver.find_element_by_xpath('//*[@id="subnav"]/div/div/h1/a').get_attribute('data-name'));
        critic_score_list.append('N/A');
        audience_score_list.append('N/A');
        positiveReviews.append('-')
        extraPositiveReviews.append('-')
        negativeReviews.append('-')
        extraNegativeReviews.append('-')

driver.close()

In [ ]:
webcrawl = pd.DataFrame(list(zip(movie_list,critic_score_list,audience_score_list,positiveReviews,extraPositiveReviews,negativeReviews,extraNegativeReviews)), columns=['Movie Name','Critic Rating','Audience Rating','Number of Positive Reviews','Number of Extra Positive Reviews','Number of Negative Reviews','Number of Extra Negative Reviews']);
display(webcrawl)

In [ ]:
# NOTE: HOW TO GET THE OUTPUT

els = $("section.movie-reviews")
$(els[0]).find("span.movie-ratings-tabs__rating").text()

$(els[0]).find("span.movie-ratings-tabs__rating").text().slice(0,3)
$(els[0]).find("span.movie-ratings-tabs__rating").text().slice(3,6)

# Output:37%76%